In [2]:
#Loading in appropriate packages
import numpy as np, matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import json
from bs4 import BeautifulSoup
import sys
from scipy import stats
from sklearn.cross_validation import *

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

%matplotlib inline

In [3]:

master = pd.read_csv('data/lahman/Master.csv')
all_batting = pd.read_csv('data/lahman/Batting.csv')
pitching = pd.read_csv('data/lahman/Pitching.csv')
smalldf = pd.read_csv('data/small.csv')

For details on the Lahman file columns, see "2.3: Pitching Table": http://seanlahman.com/files/database/readme2014.txt

pitcher_sums is just the "sums", so be sure to recalculate things like ERA and BAOpp that don't add linearly

In [9]:
master.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981,12,27,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,205,75,R,R,4/6/2004,9/28/2013,aardd001,aardsda01
1,aaronha01,1934,2,5,USA,AL,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,Hank,Aaron,Henry Louis,180,72,R,R,4/13/1954,10/3/1976,aaroh101,aaronha01
2,aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75,R,R,4/10/1962,9/26/1971,aarot101,aaronto01
3,aasedo01,1954,9,8,USA,CA,Orange,NaN,NaN,NaN,NaN,NaN,NaN,Don,Aase,Donald William,190,75,R,R,7/26/1977,10/3/1990,aased001,aasedo01
4,abadan01,1972,8,25,USA,FL,Palm Beach,NaN,NaN,NaN,NaN,NaN,NaN,Andy,Abad,Fausto Andres,184,73,L,L,9/10/2001,4/13/2006,abada001,abadan01


In [4]:
retro_to_lah = dict(zip(master['retroID'], master['playerID']))
retro_to_hand = dict(zip(master['retroID'], master['throws']))

In [5]:
pitcherIDs = smalldf['pID'].unique()#RetroIDs
L_pitcherIDs = [retro_to_lah[name] for name in pitcherIDs] #LahmanIDs

In [10]:
smalldf.head()

,bID,pID,AB,H,PA,RL,SAC,SO,TB,W,matchID,AVG,OBP,ov_AB,ov_H,ov_PA,ov_SAC,ov_SO,ov_TB,ov_W,ov_AVG,ov_OBP,ov_FACED,ov_SO_PCT,ov_W_PCT,ov_H_PCT,ovp_AB,ovp_H,ovp_PA,ovp_SAC,ovp_SO,ovp_TB,ovp_W,ovp_AVG,ovp_OBP,ovp_FACED,ovp_SO_PCT,ovp_W_PCT,ovp_H_PCT
0,schic002,benea001,9,0,9,R,1,4,0,0,schic002_benea001,0.000000,0.000000,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,3046,827,3425,46,609,1713,379,0.271504,0.352117,136,0.177810,0.110657,0.241460
1,schic002,browk001,11,1,11,R,0,6,1,0,schic002_browk001,0.090909,0.090909,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,4582,1159,4989,47,892,1974,407,0.252946,0.313891,222,0.178793,0.081579,0.232311
2,schic002,fassj001,11,2,11,L,1,5,4,0,schic002_fassj001,0.181818,0.181818,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,3006,847,3357,40,582,1587,351,0.281770,0.356866,185,0.173369,0.104558,0.252309
3,schic002,glavt001,14,3,14,L,1,3,5,0,schic002_glavt001,0.214286,0.214286,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,7561,2115,8413,102,985,3852,852,0.279725,0.352668,280,0.117081,0.101272,0.251397
4,schic002,hampm001,16,2,16,L,1,6,2,0,schic002_hampm001,0.125000,0.125000,146,22,150,14,57,30,4,0.150685,0.173333,14,0.38,0.026667,0.146667,4782,1361,5339,53,666,2355,557,0.284609,0.359243,232,0.124742,0.104327,0.254917


In [6]:
grouped =  pitching.groupby('playerID').sum().reset_index()
mask = np.in1d(grouped.playerID, L_pitcherIDs)
pitcher_sums = grouped[mask]

In [7]:
pitcher_sums.ERA = 9.*(pitcher_sums.ER)/(pitcher_sums.IPouts / 3.)
pitcher_sums.BAOpp = (pitcher_sums.H + pitcher_sums.BB + pitcher_sums.IBB + pitcher_sums.HBP) / pitcher_sums.BFP
pitcher_sums['WiP'] = pitcher_sums.W/pitcher_sums.G #win percentage
pitcher_sums['IP'] = pitcher_sums.IPouts/3.
pitcher_sums.head()

//anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http:/

,playerID,yearID,stint,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP,WiP,IP
0,aardsda01,16069,8,15,17,298,0,0,0,69,919,271,144,35,169,305,0.352155,4.230686,19,11,15,1,1346,132,152,17,10,NaN,0.050336,306.333333
7,abbotji01,21932,12,87,108,263,254,31,6,0,5022,1779,791,154,620,888,0.341284,4.252688,30,53,32,11,7211,5,880,NaN,NaN,NaN,0.330798,1674.000000
9,abbotpa01,23977,13,43,37,162,112,1,0,0,2162,682,394,101,393,496,0.349231,4.920444,17,37,21,0,3187,17,421,8,12,NaN,0.265432,720.666667
20,accarje01,20086,12,10,20,262,0,0,0,38,854,288,136,24,111,205,0.346972,4.299766,16,10,9,2,1222,115,143,5,13,NaN,0.038168,284.666667
21,aceveal01,14077,7,31,16,204,15,0,0,29,1141,333,162,48,129,282,0.310194,3.833479,6,5,28,6,1599,91,172,6,13,1,0.151961,380.333333


In [113]:
pitching.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NaN,1,2,3,3,2,0,0,78,43,23,0,11,1,NaN,7.96,NaN,NaN,NaN,0,NaN,NaN,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NaN,12,15,30,30,30,0,0,792,361,132,4,37,13,NaN,4.50,NaN,NaN,NaN,0,NaN,NaN,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NaN,0,0,1,0,0,0,0,3,8,3,0,0,0,NaN,27.00,NaN,NaN,NaN,0,NaN,NaN,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NaN,4,16,24,24,22,1,0,639,295,103,3,31,15,NaN,4.35,NaN,NaN,NaN,0,NaN,NaN,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NaN,0,1,1,1,1,0,0,27,20,10,0,3,0,NaN,10.00,NaN,NaN,NaN,0,NaN,NaN,21,NaN,NaN,NaN


In [269]:
def pitcher_sim(p1_matchups, p2_matchups, n_common):
    p1name=list(p1_matchups['pID'][:1])[0]
    p2name=list(p2_matchups['pID'][:1])[0]
    dfhands=pd.DataFrame({'RL':[list(p1_matchups['RL'][:1])[0],list(p2_matchups['RL'][:1])[0]]})
    listed= [p1name,p2name]
    two_pitcherIDs= [retro_to_lah[name] for name in listed]

    grouped2 =  pitching.groupby('playerID').sum().reset_index()
    mask2 = np.in1d(grouped2.playerID, two_pitcherIDs)
    pitchers=grouped2[mask2].reset_index()
    pitchers.ERA = 9.*(pitchers.ER)/(pitchers.IPouts / 3.)
    pitchers.BAOpp = (pitchers.H + pitchers.BB + pitchers.IBB + pitchers.HBP) / pitchers.BFP
    pitchers['WiP'] = pitchers.W/pitchers.G #win percentage
    pitchers['IP'] = pitchers.IPouts/3.
    
    
    final=pitchers.join(dfhands, how='outer')
    final['Games']=final['GS']-final['G']/2. 
    final['inningsper']=final['IP']/final['G']
    listedd=[]
    for row in final.iterrows():
        print row[1]
    #final['POS']=iterrows(final).map(lambda row: 1 if row['Games'] < 0. and row['inningsper']<4. else 0.) 
    #1 if reliever, less games started than total games

    
    score=1000
    p1=final[final.playerID==two_pitcherIDs[0]]
    p2=final[final.playerID==two_pitcherIDs[1]]
    
    
    win_diff=np.abs(int(p1['W'])-int(p2['W'])) #1 pt for each win
    loss_diff=np.abs(int(p1['L'])-int(p2['L']))/2. #1 pt each 2 losses
    wip_diff=np.abs(float(p1['WiP'])-float(p2['WiP']))/.002 #1 pt diff of winning percentage of 0.002 
    if wip_diff >100: #max= 100
        wip_diff=100
    era_diff=np.abs(float(p1['ERA'])-float(p2['ERA']))/0.02 #1 pt diff of ERA of 0.02
    if era_diff >100:
        era_diff=100 #max =100
    gp_diff=np.abs(p1['G']-p2['G'])/10. #1 pt for diff of 10 games played
    st_diff=np.abs(int(p1['GS'])-int(p2['GS']))/20. #1 pt for diff of 20 games started
    cg_diff=np.abs(int(p1['CG'])-int(p2['CG']))/20. #1 pt for diff of 20 games completed
    ip_diff=np.abs(float(p1['IP'])-float(p2['IP']))/50. #1 pt for diff of 50 innings pitched
    h_diff=np.abs(int(p1['H'])-int(p2['H']))/50. #1 pt for diff of 50 hits allowed
    sp_diff=np.abs(int(p1['SO'])-int(p2['SO']))/30. #1 pt for diff of 30 strikeouts
    bb_diff=np.abs(int(p1['BB'])-int(p2['BB']))/50. #1 pt for diff of 10 walks
    sho_diff=np.abs(int(p1['SHO'])-int(p2['SHO']))/5. #1 pt for diff of 5 shutouts
    sv_diff=np.abs(int(p1['SV'])-int(p2['SV']))/3. #1 pt for diff of 3 saves
    
    #hand_diff=10. if list(p1['RL'])[0]!=list(p2['RL'])[0] and int(p1['POS'])==0 and int(p2['POS'])==0 elif  0.
    
    
    
    
    #print #final['POS']#type(list(p1['RL'])),list(p1['RL']),hand_diff
    return final

In [270]:
pitcher_sim(smalldf[smalldf.pID=='benea001'],smalldf[smalldf.pID=='browk001'],5)

index               516
playerID      benesan01
yearID            29932
stint                16
W                   155
L                   139
G                   403
GS                  387
CG                   21
SHO                   9
SV                    1
IPouts             7516
H                  2377
ER                 1106
HR                  289
BB                  909
SO                 2000
BAOpp         0.3187412
ERA            3.973124
IBB                  52
WP                   63
HBP                  55
BK                   16
BFP               10645
GF                    6
R                  1206
SH                   17
SF                   17
GIDP                NaN
WiP           0.3846154
IP             2505.333
RL                    R
Games             185.5
inningsper     6.216708
Name: 0, dtype: object
index               937
playerID      brownke01
yearID            37923
stint                19
W                   211
L                   144
G                

,index,playerID,yearID,stint,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP,WiP,IP,RL,Games,inningsper
0,516,benesan01,29932,16,155,139,403,387,21,9,1,7516,2377,1106,289,909,2000,0.318741,3.973124,52,63,55,16,10645,6,1206,17,17,NaN,0.384615,2505.333333,R,185.5,6.216708
1,937,brownke01,37923,19,211,144,486,476,72,17,0,9769,3079,1185,208,901,2397,0.307266,3.275156,42,108,139,15,13542,1,1357,35,18,NaN,0.434156,3256.333333,R,233.0,6.700274


In [ ]:

    score=1000
    if 
    win_diff=np.abs(p1['W']-p2['W']) #1 pt for each win
    loss_diff=np.abs(p1['L']-p2['L'])/2. #1 pt each 2 losses
    wip_diff=np.abs(p1['WiP']-p2['WiP'])/.002 #1 pt diff of winning percentage of 0.002 
    if wip_diff >100: #max= 100
        wip_diff=100 
    era2_diff=np.abs(p1['ERA']-p2['ERA'])/0.02 #1 pt diff of ERA of 0.02
    if era2_diff >100:
        era2_diff=100 #max =100
    gp_diff=np.abs(p1['G']-p2['G'])/10. #1 pt for diff of 10 games played
    st_diff=np.abs(p1['GS']-p2['GS'])/20. #1 pt for diff of 20 games started
    cg_diff=np.abs(p1['CG']-p2['CG'])/20. #1 pt for diff of 20 games completed
    ip_diff=np.abs(p1['IP']-p2['IP'])/50. #1 pt for diff of 50 innings pitched
    h_diff=np.abs(p1['H']-p2['H'])/50. #1 pt for diff of 50 hits allowed
    sp_diff=np.abs(p1['SO']-p2['SO'])/30. #1 pt for diff of 30 strikeouts
    bb_diff=np.abs(p1['BB']-p2['BB'])/50. #1 pt for diff of 10 walks
    sho_diff=np.abs(p1['SHO']-p2['SHO'])/5. #1 pt for diff of 5 shutouts
    sv_diff=np.abs(p1['SV']-p2['SV'])/3. #1 pt for diff of 3 saves
    final_score=score-win_diff-loss_diff-wip_diff-era2_diff-go_diff-st_diff-cg_diff-ip_diff-h_diff-sp_diff-bb_diff-sho_diff-sv_diff  
    if np.isnan(final_score):
        return 0
    else: 
        return final_score